# Image scrapping

In [26]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from urllib.request import urlretrieve

def scrape_images_from_url(url):
    # Send a request to get the HTML content
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code != 200:
        print("Failed to retrieve the page.")
        return

    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Find all image tags
    img_tags = soup.find_all("img")

    # Create a directory to save the images
    os.makedirs("scraped_images", exist_ok=True)

    # Download and save each image
    for img_tag in img_tags:
        img_url = img_tag.get("src")
        if img_url:
            img_url = urljoin(url, img_url)
            img_name = os.path.basename(img_url)
            img_path = os.path.join("scraped_images", img_name)

            try:
                urlretrieve(img_url, img_path)
                print(f"Downloaded: {img_url}")
            except Exception as e:
                print(f"Failed to download: {img_url}")
                print(f"Error: {e}")


    

In [ ]:
url_to_scrape =
scrape_images_from_url(url_to_scrape)
 "http://xn-----6kcczalffeh6afgdgdi2apgjghic4org.xn--p1ai/"

In [141]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from urllib.request import urlretrieve
from urllib.request import urlopen

def image_scrapping(url):

    for i in range(1,50):
        if i==1:
            response = requests.get(url)

            if response.status_code != 200:
                print("There is error is getting the URL")
                return
            
            soup = BeautifulSoup(response.content,"html.parser")
            img_tags = soup.find_all("img")
            

            os.makedirs("scraped_images", exist_ok=True)
            for img_tag in img_tags:
                img_url = img_tag.get("src")
                if img_url:
                    img_url = urljoin(url, img_url)
                    img_name = os.path.basename(img_url)
                    img_path = os.path.join("scraped_images", img_name)

                    try:
                        urlretrieve(img_url, img_path)
                        print(f"Downloaded: {img_url}")
                    except Exception as e:
                        print(f"Failed to download: {img_url}")
                        print(f"Error: {e}")
        
        else:
            print(f"Scrapping from page {i} is started")
            nxt_page = f"/?page{i}"
            nex_pg_url = urljoin(url, nxt_page)
            response = requests.get(nex_pg_url)

            if response.status_code != 200:
                print("There is error is getting the URL")
                return
            
            soup = BeautifulSoup(response.content,"html.parser")
            img_tags = soup.find_all("img")
            

            os.makedirs("scraped_images", exist_ok=True)
            for img_tag in img_tags:
                img_url = img_tag.get("src")
                if img_url:
                    img_url = urljoin(url, img_url)
                    img_name = os.path.basename(img_url)
                    img_path = os.path.join("scraped_images", img_name)

                    try:
                        urlretrieve(img_url, img_path)
                        print(f"Downloaded: {img_url}")
                    except Exception as e:
                        print(f"Failed to download: {img_url}")
                        print(f"Error: {e}")
            

            

    

    
    

In [142]:
image_scrapping("http://xn-----6kcczalffeh6afgdgdi2apgjghic4org.xn--p1ai/")

Scrapping from page 2 is started
Downloaded: http://xn-----6kcczalffeh6afgdgdi2apgjghic4org.xn--p1ai/stat/1690395061
Downloaded: http://xn-----6kcczalffeh6afgdgdi2apgjghic4org.xn--p1ai/_pu/9/71822171.jpg
Downloaded: http://xn-----6kcczalffeh6afgdgdi2apgjghic4org.xn--p1ai/_pu/9/90172390.jpg
Downloaded: http://xn-----6kcczalffeh6afgdgdi2apgjghic4org.xn--p1ai/_pu/9/07196001.jpg
Downloaded: http://xn-----6kcczalffeh6afgdgdi2apgjghic4org.xn--p1ai/_pu/9/76810670.jpg
Downloaded: http://xn-----6kcczalffeh6afgdgdi2apgjghic4org.xn--p1ai/_pu/9/19341937.jpg
Downloaded: http://xn-----6kcczalffeh6afgdgdi2apgjghic4org.xn--p1ai/_pu/9/78379232.jpg
Downloaded: http://xn-----6kcczalffeh6afgdgdi2apgjghic4org.xn--p1ai/_pu/9/10499521.jpg
Downloaded: http://xn-----6kcczalffeh6afgdgdi2apgjghic4org.xn--p1ai/_pu/9/21787852.jpg
Downloaded: http://xn-----6kcczalffeh6afgdgdi2apgjghic4org.xn--p1ai/_pu/9/43201444.jpg
Downloaded: http://xn-----6kcczalffeh6afgdgdi2apgjghic4org.xn--p1ai/_pu/9/57816569.jpg
Downloaded: h

KeyboardInterrupt: 